05 Jun 2017

Convolution Test Features -- Test.

Following along: [[here](https://gist.github.com/idanori/3dadb3c1d66223170d2df4cd802da7cf)]

In [1]:
import theano

/home/wnixalo/miniconda3/envs/FAI/lib/python2.7/site-packages/theano/gpuarray/dnn.py:135: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to version 5.1.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 6021 on context None
Mapped name None to device cuda: GeForce GTX 870M (0000:01:00.0)


In [3]:
import sys, os
sys.path.insert(1, os.path.join('utils'))

%matplotlib inline
from __future__ import print_function, division

import utils
from utils import *
from IPython.display import FileLink
from shutil import copyfile, move
import pandas as pd
import matplotlib.image as mpimg

Using Theano backend.


In [4]:
path = "data/statefarm/"

In [5]:
batch_size = 16

### Save Array Function

In [6]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]
def load_carray(fname): return bcolz.open(fname)

In [7]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())
def hotone(x):
    ret = []
    for i in range(len(x)):
        for j in range(len(x[i])):
            if x[i][j] == 1:
                ret += [j]
    return ret
# concat all arrays from batch dir
def concat_batches(batches):
    return np.concatenate([batches.next() for i in range(int(batches.n/batches.batch_size+1))])

### Add Conv Layer

In [8]:
import vgg16
vggModel = Vgg16()
vggModel.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [9]:
# get index of last conv layer
lastConvIndex = -1
for i in range(len(vggModel.model.layers)):
    if (type(vggModel.model.layers[i]) == Convolution2D):
        lastConvIndex = i

print ('last conv layer index ' + str(i))

last conv layer index 37


In [10]:
convLayers = vggModel.model.layers[:lastConvIndex+1]
vggConvLayersModel = Sequential(convLayers)
vggConvLayersModel.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
                                                                   zeropadding2d_1[1][0]            
___________________________________________________________________________________________

Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
____________________________________________________________________________________________________


In [11]:
#generate an augmented train batch
imgGen = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                                  shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

augmentTrainBatch = imgGen.flow_from_directory(directory=path+'train', batch_size=64, target_size=(224,224), 
                                               shuffle=False, class_mode='categorical')

Found 19463 images belonging to 10 classes.


In [12]:
train_batches = augmentTrainBatch

# generate a new validation batch
val_batches = get_batches(path + 'valid', batch_size=batch_size, shuffle=False, 
                         target_size=(224,224))

Found 2961 images belonging to 10 classes.


In [21]:
test_batch = get_batches(path + 'test', batch_size=batch_size, shuffle=False,
                        target_size=(224,224), class_mode='binary')

Found 79726 images belonging to 1 classes.


In [22]:
val_batches.reset()
train_label_arr = onehot(train_batches.classes)
train_file_names_arr = train_batches.filenames
valid_label_arr = onehot(val_batches.classes)
valid_file_names_arr = val_batches.filenames
test_file_names_arr = test_batch.filenames

#### predict batches on pretrained model (based on conv layer of VGG16)

In [23]:
predict_test_on_VggConv = vggConvLayersModel.predict_generator(test_batch, test_batch.n)

MemoryError: 